In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import unidecode

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/carter/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [3]:
browser.visit('http://quotes.toscrape.com/')

In [4]:
data = []
authors = set()
while(browser.links.find_by_partial_text('Next ')):
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    quotes = soup.find_all('div', class_='quote')
    for quote in quotes:
        tag_list=[]
        q = quote.find('span',class_='text').text
        author = quote.find('small',class_='author').text
        tags = quote.find_all('a', class_='tag')
        for tag in tags:
            tag_list.append(tag.text)
        authors.add(author)
    #Add data to dictionary to add to MongoDB later
        data.append({'author_name':author,'quote':q,'tags': tag_list})
    browser.links.find_by_partial_text('Next ').click()

In [5]:
browser.quit()

In [6]:
author_data = []
for author in authors:
    author_ = unidecode.unidecode(author)
    chars = [' ','.','--']
    for char in chars:
        author_ = author_.replace(char,'-')

    soup = BeautifulSoup(requests.get(f"http://quotes.toscrape.com/author/{author_}").text,'html.parser')
    born = soup.find('span',class_='author-born-date').text
    description = soup.find('div', class_='author-description').text.strip()
    author_data.append({'author_name':author,'born':born,'description':description})
                                                            

In [9]:
#MongoDB
#Need to add author_data and data to mongoDB connection